In [40]:
import numpy as np
import pandas as pd
import wfdb
import scipy.io
import resampy
import neurokit2 as nk
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import os
import json
from flask_cors import CORS
import sklearn
from sklearn import preprocessing

In [41]:
def load_mat(fileName):
    data=scipy.io.loadmat(fileName)
    return data['val']

In [42]:
eeg_data = load_mat(r"C:\Users\sendm\physionet.org\files\i-care\2.0\training\0313\0313_001_004_EEG.mat")
print(eeg_data)

[[   0    0    0 ... -284 -281 -284]
 [   0    0    0 ... -301 -285 -267]
 [   0    0    0 ...   -5    0   -6]
 ...
 [   0    0    0 ...   26   35   46]
 [   0    0    0 ...  -37  -26   -6]
 [   0    0    0 ...  -96  -76  -55]]


In [43]:
def load_patient_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Parse the desired patient data from the lines
    parsed_data = {}
    for line in lines:
        
        attribute, value = line.strip().split(':')
        # Store the parsed data in the dictionary
        parsed_data[attribute.strip()] = value.strip()

    formatted_patient_info = {
    'Patient': [parsed_data['Patient']],
    'Hospital': [parsed_data['Hospital']],
    'Age': [parsed_data['Age']],
    'Sex': [parsed_data['Sex']],
    'ROSC': [parsed_data['ROSC']],
    'OHCA': [parsed_data['OHCA']],
    'Shockable Rhythm': [parsed_data['Shockable Rhythm']],
    'TTM': [parsed_data['TTM']],
    'Outcome': [parsed_data['Outcome']],
    'CPC': [parsed_data['CPC']]
    }

    return formatted_patient_info

In [44]:
import numpy as np

def reorder_eeg_channels(single_patient_eeg_data, channel_names):
    desired_channel_order = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']

    # Extract the list of channel names from the Series
    channel_names_list = channel_names.iloc[0]

    # Create a mapping dictionary for the patient's channel names
    mapping_dict = {}
    reordered_channels = []

    for channel in desired_channel_order:
        if channel in channel_names_list:
            index = channel_names_list.index(channel)
            mapping_dict[channel] = index
            reordered_channels.append(single_patient_eeg_data[index])

    # Delete channels that are not in the desired channel list using boolean indexing
    channel_names_list = [channel for channel in channel_names_list if channel in desired_channel_order]

    return reordered_channels, mapping_dict





In [45]:
def convert_patient_info(final_df):
    bad_indexes = []
    for i in final_df.index:
        if final_df['EEG'].isnull().iloc[i]:
            bad_indexes.append(i)
    clean_df = final_df.drop(bad_indexes).reset_index(drop=True)

    del clean_df['Hospital']
    del clean_df['Patient']
    del clean_df['TTM']

    clean_df['Outcome'] = clean_df['Outcome'].replace("Good", 0)
    clean_df['Outcome'] = clean_df['Outcome'].replace("Poor", 1)

    clean_df['Sex'] = clean_df['Sex'].replace("Male", 0)
    clean_df['Sex'] = clean_df['Sex'].replace("Female", 1)

    clean_df['Shockable Rhythm'] = clean_df['Shockable Rhythm'].replace("True", 0)
    clean_df['Shockable Rhythm'] = clean_df['Shockable Rhythm'].replace("False", 1)

    clean_df['OHCA'] = clean_df['OHCA'].replace("True", 0)
    clean_df['OHCA'] = clean_df['OHCA'].replace("False", 1)

    clean_df['ROSC'] = clean_df['ROSC'].replace("nan", np.nan)
    clean_df = clean_df.dropna(subset=['ROSC'])
    clean_df['Sex'] = clean_df['Sex'].replace("nan", np.nan)
    clean_df = clean_df.dropna(subset=['Sex'])
    clean_df['OHCA'] = clean_df['OHCA'].replace("nan", np.nan)
    clean_df = clean_df.dropna(subset=['OHCA'])
    clean_df['Shockable Rhythm'] = clean_df['Shockable Rhythm'].replace("nan", np.nan)
    clean_df = clean_df.dropna(subset=['Shockable Rhythm'])
    clean_df.dropna(subset=['FS'], inplace=True)

    clean_df.reset_index(drop=True, inplace=True)

    return clean_df

In [46]:
def delete_channels(clean_df):
    for i, row in clean_df.iterrows():
        eeg_data = np.array(row['EEG'])  # Convert the list to a NumPy array
        if eeg_data.shape[0] > 19:  # Check if the EEG data has more than 19 channels
            # Drop channels after the 19th channel
            clean_df.at[i, 'EEG'] = eeg_data[:19]

In [47]:
def format_data(file_path_eeg, file_path_header, file_path_patient_info):
    eeg_data = load_mat(file_path_eeg)
    
    record_header = wfdb.rdheader(file_path_header[:-4])
    fs_orig = record_header.fs
    channel_names_orig = record_header.sig_name
    patient_info = load_patient_data(file_path_patient_info)

    final_df = pd.DataFrame(data=patient_info)
    # df2 = pd.DataFrame({"EEG":eeg_data})
    # final_df['EEG'] = eeg_data

    # # Transpose the eeg_data array so that channels become rows
    # eeg_data_transposed = np.transpose(eeg_data)
    
    # # Assign transposed eeg_data to the 'EEG' column in the DataFrame
    # final_df['EEG'] = [eeg_data_transposed.tolist()]
    final_df['EEG'] = [eeg_data]
    
    
    # final_df = pd.concat([df, df2], axis=1, join='inner') 
    final_df["FS"] = fs_orig
    final_df['Channel Names'] = [channel_names_orig]


    clean_df = convert_patient_info(final_df)
    

    # reordered_eeg, map_dicts = reorder_eeg_channels(clean_df['EEG'], clean_df['Channel Names'])
    reordered_eeg = clean_df['EEG']
    
    
    clean_df['EEG'] = reordered_eeg

    del clean_df['Channel Names']
    delete_channels(clean_df)

    eeg_list = clean_df['EEG']
    fs_list = clean_df['FS']

    

    del clean_df['Outcome']
    del clean_df['EEG']
    del clean_df['FS']

    clean_df['EEG'] = eeg_list
    clean_df['FS'] = fs_list

    final_clean_df = clean_df.reset_index()
    final_clean_df['Age'] = final_clean_df['Age'].astype(int)
    final_clean_df['Sex'] = final_clean_df['Sex'].astype(int)
    final_clean_df['ROSC'] = final_clean_df['ROSC'].astype(int)
    final_clean_df['OHCA'] = final_clean_df['OHCA'].astype(int)
    final_clean_df['Shockable Rhythm'] = final_clean_df['Shockable Rhythm'].astype(int)

    return final_clean_df

In [48]:
def resample(signal, fs_orig, fs_resample):
    return resampy.resample(signal, fs_orig, fs_resample)

In [49]:
def cluster_channel_one(eeg):
    channel1 = eeg[0]
    channel2 = eeg[1]
    channel3 = eeg[2]
    channel4 = eeg[3]
    channel5 = eeg[9]
    channel6 = eeg[10]
    channel7 = eeg[15]

    sum1 = np.add(channel1,channel2)
    sum2 = np.add(sum1,channel3)
    sum3 = np.add(sum2, channel4)
    sum4 = np.add(sum3,channel5)
    sum5 = np.add(sum4,channel6)
    sum6 = np.add(sum5,channel7)

    final_sum = sum6/7
    return final_sum

def cluster_channel_three(eeg):
    channel1 = eeg[6]
    channel2 = eeg[17]

    final_sum = np.add(channel1, channel2) / 2
    return final_sum

def cluster_channel_four(eeg):
    channel1 = eeg[7]
    channel2 = eeg[8]

    sum1 = np.add(channel1, channel2)

    final_sum = sum1/2
    return final_sum

def cluster_channel_five(eeg):
    channel1 = eeg[11]
    channel2 = eeg[13]

    sum1 = np.add(channel1, channel2)

    final_sum = sum1/2
    return final_sum

def cluster_channel_six(eeg):
    channel1 = eeg[12]
    channel2 = eeg[14]

    sum1 = np.add(channel1, channel2)

    final_sum = sum1/2
    return final_sum

def cluster_eeg_channels(eeg_data):

    patient_clustered_eeg = []

    patient_clustered_eeg.append(cluster_channel_one(eeg_data))
    # patient_clustered_eeg.append(cluster_channel_two(patient_eeg)) - Cluster 2 was also noisy!
    patient_clustered_eeg.append(cluster_channel_three(eeg_data))
    patient_clustered_eeg.append(cluster_channel_four(eeg_data))
    patient_clustered_eeg.append(cluster_channel_five(eeg_data))
    patient_clustered_eeg.append(cluster_channel_six(eeg_data))

    return patient_clustered_eeg

In [50]:
def process_eeg(final_eeg_signal, fs_resample, fs_orig):

    eeg_resample = resample(np.array(final_eeg_signal), fs_orig, fs_resample)
    eeg_norm = sklearn.preprocessing.normalize(eeg_resample)
    return eeg_norm

In [51]:
def bandpassFilter(single_patient_eeg, order, start, stop, fs_resample, n_channels):
    filtered_bandpass_eeg = []
    for i in range(n_channels):  
        b, a = scipy.signal.butter(order, [start, stop], 'band', fs=fs_resample)
        single_channel_filtered_eeg = scipy.signal.lfilter(b, a, single_patient_eeg[i])
        filtered_bandpass_eeg.append(single_channel_filtered_eeg)
    return np.array(filtered_bandpass_eeg)


In [52]:
def truncate(norm_eeg, start_sample, final_sample):
    truncated_patient_data = []
    for channel_data in norm_eeg:
        truncated_channel_data = channel_data[start_sample:final_sample]
        truncated_patient_data.append(truncated_channel_data)
    return truncated_patient_data

In [53]:
def extract_features(single_patient_eeg, fs_resample):
    all_std, all_mean, all_bandpowers = [], [], []

    # Calculate the min, max, standard deviation, mean, and bandpower for the single patient's channels
    all_std.append(np.std(single_patient_eeg, axis=1))
    all_mean.append(np.mean(single_patient_eeg, axis=1))

    patient_bandpowers = []
    bandpower_df = nk.eeg_power(single_patient_eeg, sampling_rate=fs_resample)

    for _, row in bandpower_df.iterrows():
        # Get the bandpower values using the correct column names
        gamma, beta, alpha, theta, delta = row[['Hz_1_4', 'Hz_4_8', 'Hz_8_13', 'Hz_13_30', 'Hz_30_80']]
        patient_bandpowers.append([gamma, beta, alpha, theta, delta])
    all_bandpowers.append(patient_bandpowers)

    # Convert list of arrays to 2D arrays
    all_std = np.array(all_std)
    all_mean = np.array(all_mean)
    all_bandpowers = np.array(all_bandpowers) 

    # Create a DataFrame and add the features
    df = pd.DataFrame()
    for i in range(all_std.shape[1]):  # Loop over the number of channels
        df[f'Channel_{i+1}_STD'] = all_std[:, i]
        df[f'Channel_{i+1}_Mean'] = all_mean[:, i]
        
        for j in range(all_bandpowers.shape[2]):  # Loop over the number of bandpowers
            df[f'Channel_{i+1}_Bandpower_{j+1}'] = all_bandpowers[:, i, j]

    return df


In [54]:
def predict_outcome(file_path_eeg, file_path_header, file_path_patient_info, fs_resample, start_sample, final_sample, order, low_freq, upper_freq, n_channels):
    df = format_data(file_path_eeg,file_path_header,file_path_patient_info)
    
    eeg_signal = df['EEG'].tolist()
    fs_orig = df["FS"].values[0]
    
    del df["EEG"]
    del df["FS"]
    
    
    # 7th Channel was quite noisy, so this line removes 7th channel data
    del_eeg_signal = np.delete(eeg_signal, 6, axis=1)
    
    clustered_eeg = cluster_eeg_channels(del_eeg_signal[0])
    
    processed_eeg = process_eeg(clustered_eeg, fs_resample, fs_orig)
    
    truncated_eeg = truncate(processed_eeg, start_sample, final_sample)
 
    filteredBandPassEEG = bandpassFilter(truncated_eeg, order, low_freq, upper_freq, fs_resample, n_channels)

    filtered_eeg_array = np.array(filteredBandPassEEG)

    feature_df = extract_features(filtered_eeg_array, fs_resample)

    # WORK ON COMBINING PATIENT INFO AND EEG DATA
    all_features_df = pd.concat([df, feature_df], axis=1, join='inner') 

    del all_features_df['CPC']
    del all_features_df['index']
    #It shouldn't be included when making a prediction
    
    return all_features_df

    


    


In [55]:
def calc_outcome(patient_features):
    model = load_model('modelWithoutNorm.h5')
    prediction = model.predict(patient_features)
    if(prediction > .85):
        output = "It's likely that the patient will have a poor neurological outcome"
        output += "\nProbability of Poor Outcome: " + str(prediction[0][0] * 100) + "%"
        return output
    else:
        output = "It's likely that the patient will have a good neurological outcome"
        output += "\nProbability of Poor Outcome: " + str(prediction[0][0] * 100) + "%"
        return output

In [61]:
fs_resample = 100
start_sample = 42000
final_sample = 43000
n_channels = 5
order = 3
low_freq = 1
upper_freq = 49
# path_eeg = r"C:\Users\sendm\physionet.org\files\i-care\2.0\training\0313\0313_001_004_EEG.mat"
# path_header = r"C:\Users\sendm\physionet.org\files\i-care\2.0\training\0313\0313_001_004_EEG.hea"
# path_patient_info = r"C:\Users\sendm\physionet.org\files\i-care\2.0\training\0313\0313.txt"
base_path = r"C:\Users\sendm\physionet.org\files\i-care\2.0\training\0313\\"

# patient_features = predict_outcome(path_eeg, path_header, path_patient_info, fs_resample, start_sample, final_sample,order, low_freq, upper_freq, n_channels)

# outcome = calc_outcome(patient_features)
# print(outcome)

In [62]:
app = Flask(__name__)
CORS(app, resources={r"/upload": {"origins": "http://localhost:3000"}})


@app.route('/')
# Endpoint to receive JSON payload with a string data
@app.route('/upload', methods=['POST'])
def upload():
    # Get file names from the form data
    file_names = [request.form.get(f'file{i}') for i in range(1, len(request.form)+1)]
    print('Received file names:', file_names)
    
    patient_features = predict_outcome(base_path  + file_names[0], base_path + file_names[1], base_path + file_names[2], fs_resample, start_sample, final_sample,order, low_freq, upper_freq, n_channels)

    outcome = calc_outcome(patient_features)
    print(outcome)

    return {'message': outcome}

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.26:5000
Press CTRL+C to quit


Received file names: ['0313_001_004_EEG.mat', '0313_001_004_EEG.hea', '0313.txt']
1/1 [==============================] - 0s 88ms/step


192.168.1.26 - - [22/Oct/2023 15:20:30] "POST /upload HTTP/1.1" 200 -


It's likely that the patient will have a good neurological outcome
Probability of Poor Outcome: 66.34605526924133%
Received file names: ['0313_001_004_EEG.mat', '0313_001_004_EEG.hea', '0313.txt']
1/1 [==============================] - 0s 96ms/step


...

It's likely that the patient will have a good neurological outcome
Probability of Poor Outcome: 66.34605526924133%
Received file names: ['0313_001_004_EEG.mat', '0313_001_004_EEG.hea', '0313.txt']
1/1 [==============================] - 0s 89ms/step


192.168.1.26 - - [22/Oct/2023 19:21:34] "POST /upload HTTP/1.1" 200 -


It's likely that the patient will have a good neurological outcome
Probability of Poor Outcome: 66.34605526924133%
Received file names: ['0313_001_004_EEG.mat', '0313_001_004_EEG.hea', '0313.txt']
1/1 [==============================] - 0s 56ms/step


192.168.1.26 - - [22/Oct/2023 19:36:20] "POST /upload HTTP/1.1" 200 -


It's likely that the patient will have a good neurological outcome
Probability of Poor Outcome: 66.34605526924133%
